In [1]:
#########LOAD LIBRARIES###############               
import torch                                         
import os                                            
import cv2                                           
import torch.nn as nn                                
import random                                        
import torch.nn.functional as F                      
import torchvision.datasets as dset                  
import torchvision.transforms as transforms           
import torchvision.utils as utils                    
import numpy as np                                   
from torch.autograd import Variable                  
from torch import optim                              
from torch.utils.data import Dataset, DataLoader
from PIL import Image

In [2]:
#########DATA LOADER##################
class BodyPoseSet(Dataset):
    """Body pose dataset"""
    def __init__(self, root_dir='./', mode='train', transform=None):
        self.root_dir = root_dir
        self.mode = mode
        self.all_imgs, self.all_labels = self.parse_files()
        self.transform = transform

    def __len__(self):
        return len(self.all_imgs)

    def __getitem__(self, idx):
        img_name = self.all_imgs[idx]
        img_path = os.path.join(self.root_dir, img_name)
        label_name= self.all_labels[idx]
        label_path = os.path.join(self.root_dir, label_name)
        img = Image.open(img_path).convert('L')
        label = Image.open(label_path)
        
        if self.transform is not None:
            img = self.transform(img)
            label = self.transform(label)
            
        sample = {'img':img, 'label':label}

        return sample

    def parse_files(self):
        all_imgs = []
        all_labels = []
        for a in ['easy-pose']:
            for b in [i+1 for i in range(1)]:
                for c in ['Cam1','Cam2','Cam3']:
                    for d in ["{0:04}".format(i+1) for i in range(1001)]:
                        img_name = "%s/%s/%d/images/depthRender/%s/mayaProject.00%s.png" %(a,self.mode,b,c,d)
                        all_imgs.append(img_name)
                        label_name = "%s/%s/%d/images/groundtruth/%s/mayaProject.00%s.png" %(a,self.mode,b,c,d)
                        all_labels.append(label_name)
        return all_imgs, all_labels
#####################################
trans = transforms.Compose([
    transforms.Resize((250,250)),
    transforms.ToTensor(),
])

In [20]:
#########NET WORK STRUCTURE###########
class Body_Net(nn.Module):
    def __init__(self):
        super(Body_Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 64, kernel_size=5, stride=2, padding=5)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=3, stride=3)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, stride=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=5, stride=1)
        self.relu3 = nn.ReLU()
        self.conv4_class = nn.Conv2d(256, 44, kernel_size=3, stride=1)
        self.upscore1 = nn.ConvTranspose2d(44, 44, kernel_size=3, stride=1, bias=False)
        self.score_pool2 = nn.Conv2d(128, 44, kernel_size=1, stride=1)
        self.dropout = nn.Dropout2d()  # defualt = 0.5, which is used in paper
        self.upscore2 = nn.ConvTranspose2d(44, 44, kernel_size=4, stride=2, bias=False)
        self.score_pool1 = nn.Conv2d(64, 44, kernel_size=1, stride=1)
        self.upscore3 = nn.ConvTranspose2d(44, 44, kernel_size=19, stride=7, bias=False)
        self.prob = nn.Softmax2d()

    def forward(self, data):
        h = data
        h = self.relu1(self.conv1(h))
        h = self.pool1(h)
        # record pool 1
        pool1 = h
        h = self.relu2(self.conv2(h))
        h = self.pool2(h)
        # record pool 2
        pool2 = h
        h = self.relu3(self.conv3(h))
        h = self.conv4_class(h)
        h = self.upscore1(h)
        # upsample output
        upscore1 = h
        # crop pool2 and fuse with upscore1
        h =  self.score_pool2(pool2)
        h = h[:, :, 1:17, 1:17]
        score_pool2 = h
        h = upscore1 + score_pool2
        h = self.dropout(h)
        # upsample output
        h = self.upscore2(h)
        upscore2 = h
        # crop pool1 and fuse with upscore2
        h = self.score_pool1(pool1)
        h = h[:, :, 3:37, 3:37]
        score_pool1 = h
        h = upscore2 + score_pool1
        h = self.dropout(h)
        output = self.upscore3(h)
        # compute cross entropy 
        output = self.prob(output)
        output = -torch.log(output)
        return output

In [21]:
data = BodyPoseSet(transform = trans)
sample_tuple = data[0]
input_test = sample_tuple['img']
input_test = input_test.unsqueeze(0)
net = Body_Net()
output_test = net(Variable(input_test))
print output_test

Variable containing:
( 0 , 0 ,.,.) = 
  3.7855  3.7846  3.7764  ...   3.7923  3.7864  3.7805
  3.7771  3.7875  3.7795  ...   3.7814  3.7901  3.7915
  3.7816  3.7837  3.7829  ...   3.7900  3.7838  3.7886
           ...             ⋱             ...          
  3.7911  3.7874  3.7742  ...   3.7879  3.7902  3.7914
  3.7817  3.7833  3.7850  ...   3.7819  3.7798  3.7872
  3.7770  3.7849  3.7909  ...   3.7856  3.7879  3.7896

( 0 , 1 ,.,.) = 
  3.7939  3.7946  3.7876  ...   3.7865  3.7858  3.7797
  3.7824  3.7818  3.7859  ...   3.7801  3.7907  3.7785
  3.7892  3.7815  3.7788  ...   3.7832  3.7895  3.7807
           ...             ⋱             ...          
  3.7770  3.7893  3.7835  ...   3.7841  3.7884  3.7849
  3.7810  3.7821  3.7840  ...   3.7787  3.7794  3.7850
  3.7854  3.7882  3.7789  ...   3.7853  3.7845  3.7808

( 0 , 2 ,.,.) = 
  3.7769  3.7804  3.7840  ...   3.7876  3.7816  3.7852
  3.7906  3.7871  3.7763  ...   3.7870  3.7885  3.7794
  3.7869  3.7931  3.7819  ...   3.7823  3.7880